# Parameter Sweep with WaterTAP

The parameter sweep tool is an analysis tool developed by WaterTAP to explore the effect of changing model parameters or decision variables in a WaterTAP model. Using the tool requires a stable WaterTAP model. In this demonstration, we will use the RO with ERD flowsheet developed in a previous session to examing the impact of varying water permeability and water recovery on performance and technoeconomic metrics.

# Imports

We will import our working flowsheet from an existing Python file that contains all the functions necessary to build, set operating conditions, scale, initialize, and solve our model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from pyomo.environ import value, units as pyunits

from watertap.core.solvers import get_solver

from parameter_sweep import (
    LinearSample,
    ParameterSweep,
)

# Import existing flowsheet module
import RO_with_ERD as ro_erd

solver = get_solver()

In [ ]:
def build_and_solve():

    m = ro_erd.build()
    ro_erd.scale_system(m)
    ro_erd.add_costing(m)
    ro_erd.initialize_system(m)
    _ = ro_erd.solve_system(m)
    clear_output(wait=False)

    return m


m = build_and_solve()
base_lcow = value(m.fs.costing.LCOW)
base_A_comp = value(
    pyunits.convert(
        m.fs.RO.A_comp[0, "H2O"],
        to_units=pyunits.liter / pyunits.m**2 / pyunits.hour / pyunits.bar,
    )
)
base_recovery = value(m.fs.RO.recovery_vol_phase[0, "Liq"])
base_mem_area = value(m.fs.RO.area)
print(f"\nBase LCOW: {base_lcow:.2f} $/m3")

## Parameter sweep *without* the ParameterSweep tool

This cell is a demonstration of the steps the `ParameterSweep` tool is taking presented in a serial fashion:

1. Define parameter space to be swept
2. Define the results to store
3. Create an initial instance of the model
4. Unfix design variables (if necessary)
5. Fix sweep variables to sweep value
6. Solve model
7. Store results

In [ ]:
# Define number of samples and parameter ranges
num_samples = 10

# Membrane water permeability range (L/m²·h·bar)
A_comps = np.linspace(0.5, 2.5, num_samples)
A_comps = [
    a * pyunits.liter / pyunits.m**2 / pyunits.hour / pyunits.bar for a in A_comps
]

# Create lists to store results
lcows = list()
recovs = list()

# Create an initial instance of the model
m = build_and_solve()

# Unfix variables to perform the sweep
# Currently A_comp is fixed, but varying A_comp will affect performance.
# We assume the length of the membrane is fixed to our initial design
# and unfix recovery to see how it changes with A_comp
m.fs.RO.recovery_vol_phase.unfix()
m.fs.RO.length.fix(value(m.fs.RO.length))

# Perform serial sweep over A_comp values
for A in A_comps:
    m.fs.RO.A_comp[0, "H2O"].fix(A)
    results = ro_erd.solve_system(m)
    lcows.append(value(m.fs.costing.LCOW))
    recovs.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))

# Plot results
fig, ax1 = plt.subplots()
ax1.plot([value(a) for a in A_comps], lcows, ls=":", marker=".")
ax1.scatter(
    [base_A_comp],
    [base_lcow],
    color="red",
    label="Base Case",
    marker="*",
    edgecolor="k",
    s=200,
)

ax1.set(
    xlabel="Membrane Water Permeability (L/m²·h·bar)",
    ylabel="Levelized Cost of Water ($/m³)",
    ylim=(0.5, 1.25),
)
# ax1.set_xlabel("Membrane Water Permeability (L/m²·h·bar)")
# ax1.set_ylabel("Levelized Cost of Water ($/m³)")
ax1.set_title("Recovery Free\nLCOW vs. Membrane Water Permeability")
ax1.legend()

fig, ax2 = plt.subplots()

ax2.plot([value(a) for a in A_comps], recovs, ls=":", marker=".")
ax2.scatter(
    [base_A_comp],
    [base_recovery],
    color="red",
    label="Base Case",
    marker="*",
    edgecolor="k",
    s=200,
)
ax2.set_xlabel("Membrane Water Permeability (L/m²·h·bar)")
ax2.set_ylabel("RO Recovery (%)")
ax2.set_title("Recovery vs. Membrane Water Permeability")
ax2.legend()


# Building a parameter sweep in WaterTAP

A primary advantage to using the parameter sweep tool over the serial sweep demonstration in the previous cell is the ability to run more samples quickly.
Similar our serial sweep, to create a `ParameterSweep` we must provide:

- A function that builds, scales, initializes, and solves an initial flowsheet. Importantly, this function *must return the model*.
- A function that reutrns a dictionary of desired model outputs
- A function that returns the desired sweep parameters

## 1. Model build function

This is the same function we used above to perform our serial sweep above. However, we add lines to fix/unfix the variables necessary to perform our sweep. In this case, we will fix our system design by fixing the membrane length to the initial value and unfix recovery to explore how sweeping membrane permeability will impact performance.

In [ ]:
def build_and_solve():
    """
    Build and solve the RO with ERD model.
    """

    m = ro_erd.build()
    ro_erd.scale_system(m)
    ro_erd.add_costing(m)
    ro_erd.initialize_system(m)
    _ = ro_erd.solve_system(m)
    clear_output(wait=False)

    # Set system design by fixing membrane length
    m.fs.RO.length.fix(value(m.fs.RO.length))
    # Unfix recovery
    m.fs.RO.recovery_vol_phase.unfix()

    return m

## 2. Sweep outputs function

The first positional argument in this function must be our model. We instantiate an empty dictionary and then can add key/value pairs that correspond to the specific model outputs we want. In this example, we are interested in the LCOW, LCOW breakdowns, and SEC.

NOTE: The key must be a string, and the value must be a Pyomo modeling object (e.g., `Var`, `Expression`, etc.)

In [ ]:
def build_outputs(m):
    """
    Create dictionary of outputs to record from the model.
    """
    outputs = {}
    cols = [
        "fs.costing.LCOW",
        "fs.costing.SEC",
        "fs.costing.LCOW_component_direct_capex['fs.pump']",
        "fs.costing.LCOW_component_direct_capex['fs.RO']",
        "fs.costing.LCOW_component_direct_capex['fs.erd']",
        "fs.costing.LCOW_component_indirect_capex['fs.pump']",
        "fs.costing.LCOW_component_indirect_capex['fs.RO']",
        "fs.costing.LCOW_component_indirect_capex['fs.erd']",
        "fs.costing.LCOW_component_fixed_opex['fs.pump']",
        "fs.costing.LCOW_component_fixed_opex['fs.RO']",
        "fs.costing.LCOW_component_fixed_opex['fs.erd']",
        "fs.costing.LCOW_component_variable_opex['fs.pump']",
        "fs.costing.LCOW_component_variable_opex['fs.RO']",
        "fs.costing.LCOW_component_variable_opex['fs.erd']",
    ]
    for c in cols:
        outputs[c] = m.find_component(c)
    return outputs

## 3. Sweep inputs function

Like the sweep outputs function, the first argument must be our model. Then we create a `LinearSample` that defines the variable to be swept and the range of values.

NOTE: Add note on other sampling types?

In [ ]:
def build_sweep_params(m, num_samples=25):

    sweep_params = {}

    sweep_params["A_comp"] = LinearSample(m.fs.RO.A_comp, 1.0e-12, 6e-12, num_samples)

    return sweep_params

In [ ]:
from parameter_sweep import parameter_sweep

## 4. Create the `ParameterSweep` object

We can now pass these different functions to instantiate our `ParameterSweep` object.

In [ ]:
number_samples = 25
file_save = "parameter_sweep_results.csv"
results_array, results_dict = parameter_sweep(
    build_model=build_and_solve,
    build_sweep_params=build_sweep_params,
    build_outputs=build_outputs,
    num_samples=25,
    csv_results_file_name=file_save,
)

In [ ]:
import pandas as pd
parameter = "A_comp"
df = pd.read_csv(file_save)
stacked_cols = list()
colors = list()
hatch = list()
units = ["fs.RO", "fs.erd", "fs.pump"]

for u in units:
    stacked_cols.append(
        df[f"fs.costing.LCOW_component_direct_capex['{u}']"]
        + df[f"fs.costing.LCOW_component_indirect_capex['{u}']"]
    )
    stacked_cols.append(df[f"fs.costing.LCOW_component_fixed_opex['{u}']"])
    stacked_cols.append(df[f"fs.costing.LCOW_component_variable_opex['{u}']"])
    colors.extend(["#1f77b4", "#ff7f0e", "#2ca02c"])
    hatch.extend(["..", "//", "xx"])

x = df[f"# {parameter}"]

fig, ax = plt.subplots()
ax.stackplot(
    x,
    stacked_cols,
    colors=colors,
    labels=["RO", "ERD", "Pump"],
    edgecolor="black",
    hatch=hatch,
)
ax.plot(x, df["fs.costing.LCOW"], color="red", label="Total LCOW", linewidth=2)
ax.legend()
ax.axhline(0, linewidth=2, color="black")
ax.set_title(f"Parameter Sweep: {parameter}")
plt.show()


In [ ]:
stacked_cols